In [1]:
import pyspark

spark = pyspark.sql.SparkSession.builder.getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/06/16 12:07:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# Exercise 1

In [2]:
import pandas as pd
import numpy as np

np.random.seed(456)

pandas_dataframe = pd.DataFrame(
    {'languages': ['python', 'R', 'JavaScript', 'Ruby', 'C++', 'HTML']}
)
pandas_dataframe

,languages
0,python
1,R
2,JavaScript
3,Ruby
4,C++
5,HTML


In [3]:
df = spark.createDataFrame(pandas_dataframe)
df

DataFrame[languages: string]

In [4]:
df.describe()

DataFrame[summary: string, languages: string]

In [5]:
df.show(3)

+----------+
| languages|
+----------+
|    python|
|         R|
|JavaScript|
+----------+
only showing top 3 rows



In [7]:
from pydataset import data

In [8]:
df = data('mpg')

In [9]:
df = spark.createDataFrame(df)

In [10]:
df

DataFrame[manufacturer: string, model: string, displ: double, year: bigint, cyl: bigint, trans: string, drv: string, cty: bigint, hwy: bigint, fl: string, class: string]

In [11]:
df.show()

23/06/16 12:07:55 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors
+------------+------------------+-----+----+---+----------+---+---+---+---+-------+
|manufacturer|             model|displ|year|cyl|     trans|drv|cty|hwy| fl|  class|
+------------+------------------+-----+----+---+----------+---+---+---+---+-------+
|        audi|                a4|  1.8|1999|  4|  auto(l5)|  f| 18| 29|  p|compact|
|        audi|                a4|  1.8|1999|  4|manual(m5)|  f| 21| 29|  p|compact|
|        audi|                a4|  2.0|2008|  4|manual(m6)|  f| 20| 31|  p|compact|
|        audi|                a4|  2.0|2008|  4|  auto(av)|  f| 21| 30|  p|compact|
|        audi|                a4|  2.8|1999|  6|  auto(l5)|  f| 16| 26|  p|compact|
|        audi|                a4|  2.8|1999|  6|manual(m5)|

In [12]:
output = df.collect()

print(f" The {output[0][3]} {output[0][0]} {output[0][1]} has a {output[0][4]} cylinder engine")

 The 1999 audi a4 has a 4 cylinder engine


In [13]:
from pyspark.sql.functions import lit, concat

# Exercise 2

In [18]:
from pyspark.sql.functions import concat, sum, avg, min, max, count, mean


In [33]:
df.select(concat(lit("The "), df.year, lit(' '), \
                 df.manufacturer, lit(' '), df.model, lit(' has a '),\
                 df.cyl, lit(' cylinder engine')).alias('test')).show(25, False)


+-------------------------------------------------------------+
|test                                                         |
+-------------------------------------------------------------+
|The 1999 audi a4 has a 4 cylinder engine                     |
|The 1999 audi a4 has a 4 cylinder engine                     |
|The 2008 audi a4 has a 4 cylinder engine                     |
|The 2008 audi a4 has a 4 cylinder engine                     |
|The 1999 audi a4 has a 6 cylinder engine                     |
|The 1999 audi a4 has a 6 cylinder engine                     |
|The 2008 audi a4 has a 6 cylinder engine                     |
|The 1999 audi a4 quattro has a 4 cylinder engine             |
|The 1999 audi a4 quattro has a 4 cylinder engine             |
|The 2008 audi a4 quattro has a 4 cylinder engine             |
|The 2008 audi a4 quattro has a 4 cylinder engine             |
|The 1999 audi a4 quattro has a 6 cylinder engine             |
|The 1999 audi a4 quattro has a 6 cylind

In [23]:
from pyspark.sql.functions import col, expr

In [27]:
x = print(f"hello {col('model')} + whats up")

hello Column<'model'> + whats up


In [25]:
df.select(x).show(1)

TypeError: Invalid argument, not a string or column: None of type <class 'NoneType'>. For column literals, use 'lit', 'array', 'struct' or 'create_map' function.